First, import the dataset and convert it into a pandas dataframe:
----------

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

In [ ]:
ds = xr.open_dataset('MyChallengePaleo/T2m_R1_ym_1stMill.nc')
df = ds.to_dataframe()['T2m']
df

Seems like we are using a pandas MultiIndex, so let's learn a little bit about that:
-----

In [ ]:
df.index.names

Now, we want to plot some of the data. How do we do that? Let's first see what times are available... and find the first available time.
--------

In [ ]:
print(df.index.get_level_values('time'))
print(min(list(df.index.get_level_values('time'))))

We use the "xs" tool to get a slice of the data corresponding to the earliest time:
--------

In [ ]:
slice0 = df.xs(10716.0, level='time')
slice0

In [ ]:
# We could turn this into a flat dataframe, but in the end I think we do not need to do this.
# Let's keep the code around in any case.

#slice0_df = slice0.reset_index(inplace=False)
#slice0_df

Okay, this seems like something we can plot! Let's now try to figure out how to plot this.
-------
For this we will use **cartopy** (in python3, on a mac). I had a bit of difficulty finding a recipe that worked, here it is below:
```
brew extract --version=5.2.0 proj $USER/local-tap
brew install proj@5.2.0
pip3 install cartopy
```
Now we use matplotlib and cartopy to make the world-map plots.
-------
Let's first just find an example online to get a first map plot going:

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_global()
ax.stock_img()
ax.coastlines()
ax.tissot(facecolor='orange', alpha=0.4)

Let's take the MultiIndexed data that we hvae and use "unstack" to turn it into a more normal dataframe, where rows are latitude and columns are longitude.
-----
We will also reset the longitude to run from -180 to +180, in order to get it working more smoothly with the plotting.

In [ ]:
# "Unstack" the multiindex data:
unstacked = slice0.unstack(fill_value=0)
rename_to = dict()
for c in unstacked.columns :
    rename_to[c] = (c-180)%360-180+15/16

unstacked = unstacked.rename(columns=rename_to)
columns = unstacked.columns.to_list()
columns.sort()
unstacked = unstacked[columns]
unstacked


Now let's try to plot our data on a map like this.
------------

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Make the mesh grid
lon = unstacked.columns
lat = unstacked.index
m_lon, m_lat = np.meshgrid(lon, lat)

ax.contourf(m_lon, m_lat,unstacked.values,transform=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()